# **DataH Internship Program 2024-2025**

# **Desafio 4** - **Em desenvolvimento**

# **Alunos:**
## - João Paulo Cardoso.
## - Leonardo Alves Pereira.

## 1. Dados do dataset

Dados obtidos da tarefa anterior, encontrado em: https://github.com/jpscard/DataH_Internship/tree/main/Desafio%204

#### Observação sequencial de clusters

In [4]:
import pandas as pd
import numpy as np

# Carregar o dataset
df = pd.read_csv('candlestick_clusters.csv')

# definir o dataframe para análise
observations = df[['Date', 'Cluster']]

observations.head()

,Date,Cluster
0,2002-05-24,0
1,2002-05-28,0
2,2002-05-29,0
3,2002-05-30,0
4,2002-05-31,0


#### Frequência relativa de cada cluster no dataset

In [5]:
# Count the occurrences of each cluster
cluster_counts = df['Cluster'].value_counts()

# Calculate the relative frequency
relative_frequency = cluster_counts / len(df)

# Print the results
print("Cluster Counts:\n", cluster_counts)
print("\nRelative Frequency:\n", relative_frequency)

Cluster Counts:
 Cluster
0    4065
2     400
4     350
1     224
3       3
Name: count, dtype: int64

Relative Frequency:
 Cluster
0    0.806228
2    0.079334
4    0.069417
1    0.044427
3    0.000595
Name: count, dtype: float64


## 2. Configuração dos Estados Ocultos

Estados ocultos definidos:
- Alta.

- Neutro.

- Baixa.

In [6]:
# Definir estados ocultos (Alta, Baixa, Neutra)
hidden_states = ['Alta', 'Baixa', 'Neutra']
num_hidden_states = len(hidden_states)

## Pré-tratamento dos dados

In [7]:
df['Cluster'] = pd.to_numeric(df['Cluster'], errors='coerce')
print(df['Cluster'].isna().sum())  # Número de NaNs na coluna 'Cluster'
print(np.isinf(df['Cluster']).sum())  # Número de valores infinitos na coluna 'Cluster'
df = df.dropna(subset=['Cluster'])  # Remove linhas com NaN

0
0


## 3. Inicialização dos Parâmetros


- Matriz de transição (A):

  Se tiver ideia prévia de como os estados podem transitar (por exemplo, tendência de permanecer em alta ou de mudar para baixa), forneça essas suposições. Caso contrário, podemos assumir uma matriz uniforme para iniciar o processo.

- Matriz de emissões (B):
  Se houver alguma relação conhecida entre estados ocultos e clusters, indique.

- Distribuição inicial (𝜋):
  Alguma estimativa sobre os estados iniciais. Por exemplo, acredita que o mercado começa frequentemente em alta?

In [8]:
# Verificar os clusters únicos e ordenar por índice
unique_clusters = sorted(observations['Cluster'].unique())
num_clusters = len(unique_clusters)

# Inicializar matrizes
A = np.zeros((num_hidden_states, num_hidden_states))  # Matriz de transição
B = np.zeros((num_hidden_states, num_clusters))       # Matriz de emissão
pi = np.zeros(num_hidden_states)                     # Distribuição inicial

### 4. Cálculo da matriz de transição (A)

In [9]:
# Para este exemplo, vamos criar uma sequência fictícia de estados ocultos correspondente ao seu dataset.
# No mundo real, isso seria aprendido com o algoritmo Baum-Welch.

# Simular estados ocultos para cada cluster (use regras ou análises para ajustar isso)
# Exemplo fictício: estados alternados com base nos clusters
observations['hidden_states'] = np.random.choice(hidden_states, len(observations), replace=True)

# Calcular transições entre estados ocultos
for (s1, s2) in zip(observations['hidden_states'][:-1], observations['hidden_states'][1:]):
    idx_s1 = hidden_states.index(s1)
    idx_s2 = hidden_states.index(s2)
    A[idx_s1, idx_s2] += 1

# Normalizar para obter probabilidades
A = A / A.sum(axis=1, keepdims=True)

### 5. Cálculo da matriz de emissões (B)

In [10]:
# Calcular as emissões (frequência relativa de clusters dados os estados ocultos)
for state, cluster in zip(observations['hidden_states'], observations['Cluster']):
    state_idx = hidden_states.index(state)
    cluster_idx = unique_clusters.index(cluster)
    B[state_idx, cluster_idx] += 1

# Normalizar para obter probabilidades
B = B / B.sum(axis=1, keepdims=True)

### 6. Distribuição inicial (pi)

In [11]:
# Contar o primeiro estado oculto em cada dia e calcular a frequência relativa
first_states = observations.groupby('Date')['hidden_states'].first().values
for state in first_states:
    state_idx = hidden_states.index(state)
    pi[state_idx] += 1

# Normalizar para obter probabilidades
pi = pi / pi.sum()

### 7. Exibir resultados

In [12]:
# --- Exibir resultados ---
print("Matriz de Transição (A):")
print(A)

print("\nMatriz de Emissão (B):")
print(B)

print("\nDistribuição Inicial (pi):")
print(pi)

Matriz de Transição (A):
[[0.32002384 0.33730632 0.34266985]
 [0.32595694 0.33911483 0.33492823]
 [0.35245417 0.31933767 0.32820816]]

Matriz de Emissão (B):
[[8.00953516e-01 3.69487485e-02 8.82002384e-02 5.95947557e-04
  7.33015495e-02]
 [8.11117753e-01 4.96114764e-02 7.23251644e-02 5.97728631e-04
  6.63478781e-02]
 [8.06623300e-01 4.67179184e-02 7.74689533e-02 5.91366056e-04
  6.85984624e-02]]

Distribuição Inicial (pi):
[0.33280444 0.33181277 0.33538278]
